In [1]:
'''
Author: Conny Zhou
Email: junyi.zhou@emory.edu
Last Updated: 01/25/2023
'''

import boto3
import pandas as pd

def download_file_from_s3(bucket, object_name, local_file_name):
    """
    Download a file from S3 to the local file system.

    :param bucket: Name of the S3 bucket
    :param object_name: S3 object name
    :param local_file_name: Local file name to save the downloaded file
    """
    s3_client = boto3.client('s3')
    s3_client.download_file(bucket, object_name, local_file_name)

bucket_name = 'myukdata'
s3_file_names = ['Bills/BillLatestStage_Date/BillsLatestStage_Date.csv', 'Bills/BillLatestStage_ID/BillsLatestStage_ID.csv']
local_file_names = ['BillsLatestStage_Date.csv', 'BillsLatestStage_ID.csv']

# Download files
for s3_file, local_file in zip(s3_file_names, local_file_names):
    download_file_from_s3(bucket_name, s3_file, local_file)

# Read into pandas DataFrames
date = pd.read_csv(local_file_names[0])
id = pd.read_csv(local_file_names[1])




In [2]:
print(date.info())
print(id.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3431 entries, 0 to 3430
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   billId                      3431 non-null   int64 
 1   shortTitle                  3431 non-null   object
 2   currentHouse                3431 non-null   object
 3   originatingHouse            3431 non-null   object
 4   lastUpdate                  3431 non-null   object
 5   billWithdrawn               50 non-null     object
 6   isDefeated                  3431 non-null   bool  
 7   billTypeId                  3431 non-null   int64 
 8   introducedSessionId         3431 non-null   int64 
 9   includedSessionIds          3431 non-null   object
 10  isAct                       3431 non-null   bool  
 11  currentStage.id             3431 non-null   int64 
 12  currentStage.stageId        3431 non-null   int64 
 13  currentStage.sessionId      3431 non-null   int6

In [7]:
pd.set_option('display.max_columns', None)
df.head()

,billId,shortTitle_x,currentHouse_x,originatingHouse_x,lastUpdate_x,billWithdrawn_x,isDefeated_x,billTypeId_x,introducedSessionId_x,includedSessionIds_x,isAct_x,currentStage.id_x,currentStage.stageId,currentStage.sessionId_x,currentStage.description_x,currentStage.abbreviation_x,currentStage.house_x,currentStage.stageSittings_x,currentStage.sortOrder_x,member.memberId,longTitle,summary,petitioningPeriod,petitionInformation,agent,shortTitle_y,currentHouse_y,originatingHouse_y,lastUpdate_y,billWithdrawn_y,isDefeated_y,billTypeId_y,introducedSessionId_y,includedSessionIds_y,isAct_y,currentStage.id_y,currentStage.sessionId_y,currentStage.description_y,currentStage.abbreviation_y,currentStage.house_y,currentStage.stageSittings_y,currentStage.sortOrder_y,member,sortOrder,organisation.name,organisation.url
0,29,Alcohol Labelling Bill [HL],Lords,Lords,2007-10-10T09:11:00,NaN,False,2,20,[20],False,146,2,20,2nd reading,2R,Lords,"[{'id': 5, 'stageId': 2, 'billStageId': 146, '...",2,2570.0,To make provision for the labelling of alcohol...,NaN,NaN,NaN,NaN,Alcohol Labelling Bill [HL],Lords,Lords,2007-10-10T09:11:00,NaN,False,2.0,20.0,[20],False,146.0,20.0,2nd reading,2R,Lords,"[{'id': 5, 'stageId': 2, 'billStageId': 146, '...",2.0,NaN,NaN,NaN,NaN
1,35,Development Orders (Microgeneration) (formerly...,Lords,Lords,2007-10-13T16:36:00,NaN,False,2,20,[20],False,170,3,20,Committee stage,CS,Lords,"[{'id': 25, 'stageId': 3, 'billStageId': 170, ...",3,3271.0,Make provision for a review of permitted devel...,NaN,NaN,NaN,NaN,Development Orders (Microgeneration) (formerly...,Lords,Lords,2007-10-13T16:36:00,NaN,False,2.0,20.0,[20],False,170.0,20.0,Committee stage,CS,Lords,"[{'id': 25, 'stageId': 3, 'billStageId': 170, ...",3.0,NaN,NaN,NaN,NaN
2,63,Royal Commission (Slavery) Bill [HL],Lords,Lords,2007-10-13T17:02:00,NaN,False,2,20,[20],False,12981,2,20,2nd reading,2R,Lords,[],2,3364.0,Make provision for the establishment of a Roya...,NaN,NaN,NaN,NaN,Royal Commission (Slavery) Bill [HL],Lords,Lords,2007-10-13T17:02:00,NaN,False,2.0,20.0,[20],False,12981.0,20.0,2nd reading,2R,Lords,[],2.0,NaN,NaN,NaN,NaN
3,37,European Union (Implications of Withdrawal) Bi...,Lords,Lords,2007-10-13T17:36:00,NaN,False,2,20,[20],False,177,2,20,2nd reading,2R,Lords,"[{'id': 32, 'stageId': 2, 'billStageId': 177, ...",2,3153.0,Establish a Committee of Inquiry into the impl...,NaN,NaN,NaN,NaN,European Union (Implications of Withdrawal) Bi...,Lords,Lords,2007-10-13T17:36:00,NaN,False,2.0,20.0,[20],False,177.0,20.0,2nd reading,2R,Lords,"[{'id': 32, 'stageId': 2, 'billStageId': 177, ...",2.0,NaN,NaN,NaN,NaN
4,54,Light Bulb (Regulation) Bill,Commons,Commons,2007-10-19T12:28:00,NaN,False,8,20,[20],False,12972,7,20,2nd reading,2R,Commons,[],2,1412.0,NaN,NaN,NaN,NaN,NaN,Light Bulb (Regulation) Bill,Commons,Commons,2007-10-19T12:28:00,NaN,False,8.0,20.0,[20],False,12972.0,20.0,2nd reading,2R,Commons,[],2.0,NaN,NaN,NaN,NaN


In [4]:
#Initial attempt. You can ignore this
df = pd.merge(date, id, on=['billId'])

In [8]:
df['shortTitle_x']
df['shortTitle_y']
# Count of equal items
equal_count = (df['shortTitle_x'] == df['shortTitle_y']).sum()
print('Number of equal items:', equal_count)

# Count of unequal items
unequal_count = (df['shortTitle_x'] != df['shortTitle_y']).sum()
print('Number of unequal items:', unequal_count)

Number of equal items: 3381
Number of unequal items: 607


In [15]:
df[df['shortTitle_x'] != df['shortTitle_y']]['shortTitle_y'].info()

<class 'pandas.core.series.Series'>
Index: 607 entries, 32 to 3987
Series name: shortTitle_y
Non-Null Count  Dtype 
--------------  ----- 
0 non-null      object
dtypes: object(1)
memory usage: 9.5+ KB


In [35]:
#This here is the new outerjoin method while replacing some of the null value with the other column
result = date.merge(id, on='billId', how='outer',suffixes=('_left', '_right'))
result.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4088 entries, 0 to 4087
Data columns (total 46 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   billId                            4038 non-null   float64
 1   shortTitle_left                   4038 non-null   object 
 2   currentHouse_left                 4038 non-null   object 
 3   originatingHouse_left             4038 non-null   object 
 4   lastUpdate_left                   4038 non-null   object 
 5   billWithdrawn_left                50 non-null     object 
 6   isDefeated_left                   4038 non-null   object 
 7   billTypeId_left                   4038 non-null   float64
 8   introducedSessionId_left          4038 non-null   float64
 9   includedSessionIds_left           4038 non-null   object 
 10  isAct_left                        4038 non-null   object 
 11  currentStage.id_left              4038 non-null   float64
 12  curren

In [36]:
# Loop through each column and apply the logic
for column in result.columns:
    # Check if the column ends with '_left'
    if column.endswith('_left'):
        # Extract the base name of the column (without '_left')
        base_column_name = column[:-5]
        # Replace NaN in the '_left' column with values from the '_right' column
        result[column] = result[column].fillna(result[base_column_name + '_right'])
        # Drop the '_right' column
        result.drop(base_column_name + '_right', axis=1, inplace=True)
        # Rename '_left' column to original name
        result.rename(columns={column: base_column_name}, inplace=True)


result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4088 entries, 0 to 4087
Data columns (total 29 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   billId                      4038 non-null   float64
 1   shortTitle                  4088 non-null   object 
 2   currentHouse                4088 non-null   object 
 3   originatingHouse            4088 non-null   object 
 4   lastUpdate                  4088 non-null   object 
 5   billWithdrawn               52 non-null     object 
 6   isDefeated                  4088 non-null   bool   
 7   billTypeId                  4088 non-null   float64
 8   introducedSessionId         4088 non-null   float64
 9   includedSessionIds          4088 non-null   object 
 10  isAct                       4088 non-null   bool   
 11  currentStage.id             4088 non-null   float64
 12  currentStage.stageId        4038 non-null   float64
 13  currentStage.sessionId      4088 

In [37]:
#Remove rows whose billId is NaN
result = result[result['billId'].notna()]
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4038 entries, 0 to 4037
Data columns (total 29 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   billId                      4038 non-null   float64
 1   shortTitle                  4038 non-null   object 
 2   currentHouse                4038 non-null   object 
 3   originatingHouse            4038 non-null   object 
 4   lastUpdate                  4038 non-null   object 
 5   billWithdrawn               50 non-null     object 
 6   isDefeated                  4038 non-null   bool   
 7   billTypeId                  4038 non-null   float64
 8   introducedSessionId         4038 non-null   float64
 9   includedSessionIds          4038 non-null   object 
 10  isAct                       4038 non-null   bool   
 11  currentStage.id             4038 non-null   float64
 12  currentStage.stageId        4038 non-null   float64
 13  currentStage.sessionId      4038 

In [42]:
print(date['billId'].count())
print(date['billId'].nunique())
print(id['billId'].count())
print(id['billId'].nunique())
#This means that for the id dataset, there are duplicate rows used for single bill with multiple sponsors

3431
3431
3988
3381


In [43]:
print(result['billId'].count())
print(result['billId'].nunique())

4038
3431


In [44]:
#Write the result to csv
result.to_csv('demo.csv', index=False)

In [ ]:
#Save the file to S3
import boto3
import io

def upload_df_to_s3(df, bucket, object_name):
    """
    Upload a DataFrame to an S3 bucket as CSV.

    :param df: DataFrame to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name
    :return: True if the DataFrame was uploaded, else False
    """
    # Create a buffer
    csv_buffer = io.StringIO()
    df.to_csv(csv_buffer, index=False)

    # Move to the start of the buffer
    csv_buffer.seek(0)

    # Upload the buffer content to S3
    s3_client = boto3.client('s3')
    try:
        s3_client.put_object(Bucket=bucket, Key=object_name, Body=csv_buffer.getvalue())
    except ClientError as e:
        logging.error(e)
        return False
    return True

bucket_name = 'myukdata'
object_name = 'Bills/BillsLatestStage_Combined/BillsLatestStage_Combined.csv'
upload_df_to_s3(df, bucket_name, object_name)
